In [1]:
from utils.paths import *
from utils.dataset import *


In [2]:
def get_df_lists(gps_coords, radius, grid_size):
    print('Getting geometry')
    array, observer_pixel, observer_height  = get_masked_data(gps_coords, radius, grid_size, path)
    global forepeaks_df
    global summits_df
    peaks_df, forepeaks_df, summits_df = get_peaks_forepeaks( array, observer_pixel, observer_height, radius, grid_size)
    global mdf_list
    global tdf_list
    global amdf_list 
    mdf_list, tdf_list, amdf_list  = [], [], []
    for month in np.arange(1,13):
        # td = datetime.timedelta(days = interval)
        # final_date = start_date + n_intervals * td
        year = 2021
        day = 1
        date = {'year':year,'month':month,'day':day}
        date = datetime.date(year = date['year'], month = date['month'], day = date['day'])
        mdf_,tdf_,amdf_  = get_sun_data(gps_coords, observer_height, peaks_df, date)
        mdf_list.append(mdf_)
        tdf_list.append(tdf_)
        amdf_list.append(amdf_)
    # return mdf_list, tdf_list, amdf_list 

def make_solmap(month = 6): 
    mdf = mdf_list[month-1]
    tdf = tdf_list[month-1]

    peak_lines = dict(
                    type = 'scatter',
                    x = mdf.bearing,
                    y = mdf.peak_angle,
                    line = dict(color = 'gold'),
                    mode = 'none',
                    stackgroup = 'sun',
                    fillcolor = 'black'
                    # fill = 'tozeroy'
                    )
    diff_lines = dict(
                    type = 'scatter',
                    x = mdf.bearing,
                    y = mdf.el_diff,
                    line = dict(color = 'gold'),
                    mode = 'none',
                    stackgroup = 'sun',
                    fillcolor = 'gold'
                    # fill = 'tozeroy'
                    )
    sun_line = dict(
                    type = 'scatter',
                    x = mdf.loc[ mdf.daylight == 'day' ].bearing,
                    y = mdf.loc[ mdf.daylight == 'day' ].elevation,
                    line = dict(
                        color = 'white',
                        width = 0.5),
                )
    
    ticks, annotations = get_annotations(tdf)

    pio.templates.default = "simple_white"

    fig = go.Figure()

    fig.add_traces([
        peak_lines,
        diff_lines,
        sun_line,
    ])
    fig.add_traces(
        ticks
    )
    max_y = max( [ max( [mdf_.elevation.max(), mdf_.peak_angle.max()] ) for mdf_ in mdf_list ] )

    fig.update_layout( 
        annotations = annotations,
        xaxis = dict(
            tickmode = 'array',
            tickvals = [0, 22.5, 45.0, 67.5, 90.0, 112.5, 135.0, 157.5, 180.0, 202.5, 225.0, 247.5, 270.0, 292.5, 315, 337.5, 360] ,
            ticktext = ['N', 'ENE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NWN', 'N'],
            range = [45, 315],
        ),
        yaxis = dict(
            range = [0, max_y],
            anchor = 'free',
            position = 0.5,
            visible = False,
            scaleanchor = 'x',
            scaleratio = 0.6
        ),
        showlegend=False,
        autosize=True,
        height=600,
        # margin={'t': 50},
        # margin=dict(
        #     l=50,
        #     r=50,
        #     b=100,
        #     t=100,
        #     pad=4
        # ),
        paper_bgcolor="white",
        font = dict( family = 'verdana', size = 12 )
    )
    return fig

def get_annotations(tdf):
    annotations = []
    ticks = []
    gdf = tdf.loc[tdf.elevation > 0].groupby('time')
    for hour, df in gdf:
        df = df.sort_values('date').reset_index()
        # print(df.head)
        x = float( df['azimuth' ] )
        y = float( df['elevation' ] )
        text = hour
        grad = float( df.grad )
        ticks.append(dict(
            type = 'scatter',
            x=[ x, x + 2 * grad],
            y=[ y, y - 2] ,
            line = dict( color="white", width=1), 
            # fill = 'toself',
            marker = None,
            mode = 'lines',
        ))
        
        # markers.append(make_marker_dict(x,y,hour))
        annotations.append( dict(
            text = str(text) + ':00',
            x = x,
            y = y,
            xanchor = 'center',
            yanchor = 'middle',
            xshift = 15 * grad,
            yshift = -20,
            showarrow = False,
            font = dict(color = 'white'),
            opacity = 1
        )    )
    return ticks, annotations


In [3]:
gps_coords = 46.5026174, 8.3203729
# global radius, grid_size
radius = 15000
grid_size = 10
path = '/Users/george-birchenough/sunmap_rasters/Switzerland_DEM_10m.tif'
get_df_lists(gps_coords, radius, grid_size)
fig = make_solmap(6)


Getting geometry


/Users/george-birchenough/sunmap/utils/paths.py:227: RuntimeWarning: divide by zero encountered in true_divide
  elevations = np.arctan( heights / distances ) * 180/np.pi


In [4]:
summits_df

,bearing,peak_angle,peak_height,peak_distance
0,92,20,1714,840
0,171,24,2089,1510
0,326,18,2487,3160


In [5]:
for column in forepeaks_df.columns:
    print(column)
    d1 = dict(
        type='scatter',
        x = forepeaks_df.index,
        y = forepeaks_df[column],
        mode = 'markers',
        marker=dict(
            color='white',
            size=2
        ),
    )
    fig.add_trace(d1)

forepeak_angle_0


In [6]:
fig.add_trace(
    dict(
        type = 'scatter',
        x = summits_df['bearing'],
        y = summits_df['peak_angle'],
        text = summits_df['peak_height'],
        textfont = dict(
            size = 12,
            color = 'white'
        ),
        mode = 'markers+text',
        marker=dict(
            color='blue',
            size=2
        ),
    )
)
fig.show()

In [7]:
summits_df['peak_height'].to_list()

[1714, 2089, 2487]